In [1]:
import os
from IPython.display import JSON

from langchain.chat_models import init_chat_model

ds_llm = init_chat_model(
    "deepseek/deepseek-chat-v3-0324:free", 
    model_provider="openai",
    base_url=os.getenv("OPENROUTER_BASE_URL"), 
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
)
ds_llm.invoke("Tell me a joke about cats")

AIMessage(content="Here’s a purr-fect cat joke for you:  \n\n**Why did the cat sit on the computer?**  \nBecause it wanted to keep an *eye on the mouse*!  \n\n(🐱🖱️😂) Let me know if you'd like another one!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 11, 'total_tokens': 71, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1749966810-K5q2ySErmWsv7mEiG243', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--5291077f-592d-491c-8090-6b11b1c03ebf-0', usage_metadata={'input_tokens': 11, 'output_tokens': 60, 'total_tokens': 71, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field
import pprint
#  Pydantic 
class Joke(BaseModel):
    """一个笑话"""
    setup: str = Field(description="笑话的铺垫部分")
    punchline: str = Field(description="笑话的包袱部分")
    rating: Optional[int] = Field(
        default=None,
        description="笑话的评分，范围是1到10"
    )

structured_llm = ds_llm.with_structured_output(Joke)

output = structured_llm.invoke("讲个苏联笑话")



In [4]:
pprint.pprint(output)

Joke(setup='商店排长队买肉，有人抱怨道：“这帮政府官员真是无能！”', punchline='“同志，您对政府还有什么意见吗？”\n“没有，我对政府非常满意——我刚才只是在念自己的购物清单。”', rating=4)


In [6]:
from typing_extensions import Annotated, TypedDict

# TypedDict
class Joke2(TypedDict):
    """一个笑话""" 
    setup: Annotated[str,...,"笑话的铺垫部分"]
    punchline: Annotated[str,...,"笑话的包袱部分"]
    rating: Annotated[Optional[int],...,"笑话的评分，范围是1到10"]

structured_llm = ds_llm.with_structured_output(Joke2)
output = structured_llm.invoke("讲个苏联笑话")



In [7]:
pprint.pprint(output)

{'punchline': '同志，您听说过关于苏联汽车的笑话吗？现在你可以申请排队领取了！',
 'rating': 9,
 'setup': '为什么苏联工厂要生产左轮手枪？'}


In [9]:
json_schema = {
    "title":"Joke",
    "description":"一个笑话",
    "type":"object",
    "properties":{
        "setup":{
            "type":"string",
            "description":"笑话的铺垫部分"
        },
        "punchline":{
            "type":"string",
            "description":"笑话的包袱部分"
        },
        "rating":{
            "type":"integer",
            "description":"笑话的评分，范围是1到10"
        },
    },
    "required":["setup","punchline"]
}
structured_llm = ds_llm.with_structured_output(json_schema)
output = structured_llm.invoke("讲个苏联笑话")
pprint.pprint(output)

{'punchline': "'在共产主义社会，还会有警察吗？'学生们异口同声：'不会！因为到那时，所有公民都已学会自己逮捕自己。'",
 'setup': '在苏联同志的公民课上，老师提问：'}


In [15]:
from typing import Union 

class CoversationalResponse(BaseModel):
    """以对话的形式回应"""
    response: str = Field(description="对用户问题的回应")

class FinalResponse(BaseModel):
    final_output: Union[Joke, CoversationalResponse]

structured_llm = ds_llm.with_structured_output(FinalResponse)
output = structured_llm.invoke("讲个苏联笑话")
pprint.pprint(output)

FinalResponse(final_output=CoversationalResponse(response='好的，我来讲一个经典的苏联笑话：\n\n一位克格勃军官在红场拦住一个拎着沉重行李箱的市民：\n- 箱子里是什么？\n- 200克香肠、面包和黄油。\n军官打开箱子检查，发现全是伏特加：\n- 你竟敢说谎？这是伏特加！\n市民冷静回答：\n- 您看，这就是我们社会主义的成就——您问我有什么，我说的是理想，您看到的却是现实！'))


In [17]:
for chunk in structured_llm.stream("讲个苏联笑话"):
    print(chunk)

final_output=CoversationalResponse(response='好的，这里有一个经典的苏联笑话：\n\n**笑话内容**:\n勃列日涅夫带着母亲参观他的豪宅，展示奢华的生活。母亲看完后忧心忡忡地问："孩子，这一切都很好……可是如果共产党回来了怎么办？"\n\n**解读**:\n1. 讽刺勃列日涅夫时代的腐败：领导人所享受的特权与共产主义理想背道而驰；\n2. 隐含「共产党早已变质」的黑色幽默——连领导人的母亲都觉得现在的政权已不再是真正的共产党；\n3. 多层反讽：本该代表无产阶级的执政者，反而成了被人民警惕的「反动派」。\n\n***注意事项***\n• 这类笑话诞生于特定历史背景，部分内容可能较为尖锐\n• 当代俄罗斯对苏联时期的评价复杂，笑话的解读也存在不同视角')


In [22]:
from langchain_core.prompts import ChatPromptTemplate 

system = """你是一位非常搞笑的喜剧演员。你的专长是苏联笑话。
请给出一个笑话，包括setup部分和最后的punchline部分。punchline部分是对setup部分的回应.

这里有一些例子:

example_user: 讲一个苏联笑话
example_assistant: {{
    "punchline": "因为它们是世界上最快的。你可以在一小时内走完一年的路程！",
    "rating": 8,
    "setup": "为什么苏联的汽车很糟糕？"
  }},

example_user: 再讲一个苏联笑话
example_assistant: {{
    "punchline": "在资本主义社会，人剥削人；在社会主义社会，则完全相反。",
    "rating": 10,
    "setup": "资本主义和社会主义有什么区别？"
  }},

example_user: 还不够
example_assistant: {{
    "punchline": "因为他们知道，只有在排队的时候才能遇到所有认识的人。",
    "rating": 7,
    "setup": "为什么苏联人喜欢排队？"
}}
"""
prompt = ChatPromptTemplate.from_messages([("system",system),("human", "{input}")])

few_shot_structured_llm = prompt | structured_llm
few_shot_structured_llm.invoke("今天有什么好笑的?")

FinalResponse(final_output=Joke(setup='为什么苏联时期的天气预报总是准确的？', punchline="因为他们只报一种天气：'明天全国普遍寒冷，部分地区有希望'。", rating=9))

In [24]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("讲一个苏联笑话", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{
            "name": "joke",
            "args":{
                "setup":"为什么苏联的汽车很糟糕？",
                "punchline": "因为它们是世界上最快的。你可以在一小时内走完一年的路程！",
                "rating": 8,
            },
            "id": "1",
        }]
    ),
    ToolMessage("", tool_call_id="1"),
    HumanMessage("再讲一个苏联笑话", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{
            "name": "joke",
            "args":{
                "punchline": "在资本主义社会，人剥削人；在社会主义社会，则完全相反。",
                "rating": 10,
                "setup": "资本主义和社会主义有什么区别？"
            },
            "id": "2",
        }]
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("再讲一个苏联笑话", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[{
            "name": "joke",
            "args":{
                "punchline": "因为他们知道，只有在排队的时候才能遇到所有认识的人。",
                "rating": 7,
                "setup": "为什么苏联人喜欢排队？"
            },
            "id": "3",
        }]
    ),
    ToolMessage("", tool_call_id="3"),
]

system = """你是一位非常搞笑的喜剧演员。你的专长是苏联笑话。
请给出一个笑话，包括setup部分和最后的punchline部分。punchline部分是对setup部分的回应."""

prompt = ChatPromptTemplate.from_messages(
    [("system",system),("placeholder","{examples}"),("human","{input}")]
)

few_shot_tool_calling = prompt | structured_llm 

few_shot_tool_calling.invoke("今天有什么好笑的?")

FinalResponse(final_output=Joke(setup="一位苏联公民走进商店想买汽车，销售员告诉他需要排队等10年。这位公民问道：'是上午来取车还是下午来取车？'", punchline="销售员惊讶地回答：'同志，这可是10年后的事！' 公民认真地解释：'因为那天上午水管工要来我家...'", rating=4))

In [36]:
structured_llm = ds_llm.with_structured_output(None, method="json_mode")

structured_llm.invoke("讲一个苏联笑话. 使用JSON格式返回结果, 确保结果包含'setup'和'punchline'字段")

{'setup': "一位同志在红场散步时抱怨道：'这个政府真差劲！'",
 'punchline': '结果他被逮捕了——不是因为批评政府，而是因为泄露国家机密。'}

In [26]:
structured_llm = ds_llm.with_structured_output(Joke, include_raw=True)

structured_llm.invoke("Tell me a joke about cats")

{'raw': AIMessage(content='{ "punchline": "Why don\'t cats play poker in the jungle?", "rating": 10, "setup": "Too many cheetahs!" }', additional_kwargs={'parsed': Joke(setup='Too many cheetahs!', punchline="Why don't cats play poker in the jungle?", rating=10), 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 9, 'total_tokens': 44, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek/deepseek-chat-v3-0324:free', 'system_fingerprint': None, 'id': 'gen-1749978225-AiiZJvSYAz7QzgqagIcT', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--f480cefb-2211-4a64-863d-87fecd391f73-0', usage_metadata={'input_tokens': 9, 'output_tokens': 35, 'total_tokens': 44, 'input_token_details': {}, 'output_token_details': {}}),
 'parsed': Joke(setup='Too many cheetahs!', punchline="Why don't cats play poker in the jungle?", rating=10),
 'parsing_error': None}

In [29]:
from typing import List 

from langchain_core.output_parsers  import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Person(BaseModel):
    """个人信息"""

    name: str = Field(..., description="人的姓名")
    height_in_meters: float = Field(..., description="人的身高（米）")

class People(BaseModel):
    """文本中所有人的信息"""
    people: List[Person] = Field(..., description="文本中所有人的信息")
parser = PydanticOutputParser(pydantic_object=People)
prompt = ChatPromptTemplate.from_messages([
    ("system","回答用户的查询。将输出用json标签包裹 \n {format_instructions}"),
    ("human","{query}"),
]).partial(format_instructions=parser.get_format_instructions())

query = "大傻春是个身高一米六六的18岁纯情男大."

print(prompt.invoke({"query": query}).to_string())

System: 回答用户的查询。将输出用json标签包裹 
 The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Person": {"description": "个人信息", "properties": {"name": {"description": "人的姓名", "title": "Name", "type": "string"}, "height_in_meters": {"description": "人的身高（米）", "title": "Height In Meters", "type": "number"}}, "required": ["name", "height_in_meters"], "title": "Person", "type": "object"}}, "description": "文本中所有人的信息", "properties": {"people": {"description": "文本中所有人的信息", "items": {"$ref": "#/$defs/Person"}, "title": "People", "type": "array"}}, "required": ["people"]}
```
Human: 大傻春是个身高一米六六的18岁纯情男大.


In [31]:
chain = prompt | ds_llm | parser 
chain.invoke({"query":query})

People(people=[Person(name='大傻春', height_in_meters=1.66)])

In [32]:
import json
import re
from typing import List

from langchain_core.messages import AIMessage 
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "回答用户的查询。用json格式标签输出"
            "按照如下格式: \`\`\`json{schema}\n\`\`\`."
            "一定要确保你的回答在\`\`\`json 和 \`\`\`之间."
        ),
        ("human", "{query}"),
    ]
).partial(schema=People.schema())

def extract_json(message: AIMessage)->List[dict]:
    text = message.content
    parttern = r"\`\`\`json(.*?)\`\`\`"
    matches = re.findall(parttern, text, re.DOTALL)

    try:
        return [json.loads(match.strip()) for match in matches]
    except Exception:
        raise ValueError(f"提取信息失败: {message}")

query = "大傻春是个身高一米六六的18岁纯情男大."

print(prompt.invoke({"query": query}).to_string())

chain = prompt | ds_llm | extract_json
chain.invoke({"query":query})

<>:14: SyntaxWarning: invalid escape sequence '\`'
<>:15: SyntaxWarning: invalid escape sequence '\`'
<>:14: SyntaxWarning: invalid escape sequence '\`'
<>:15: SyntaxWarning: invalid escape sequence '\`'
C:\Users\joe\AppData\Local\Temp\ipykernel_29444\2998752214.py:14: SyntaxWarning: invalid escape sequence '\`'
  "按照如下格式: \`\`\`json{schema}\n\`\`\`."
C:\Users\joe\AppData\Local\Temp\ipykernel_29444\2998752214.py:15: SyntaxWarning: invalid escape sequence '\`'
  "一定要确保你的回答在\`\`\`json 和 \`\`\`之间."
C:\Users\joe\AppData\Local\Temp\ipykernel_29444\2998752214.py:19: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  ).partial(schema=People.schema())


System: 回答用户的查询。用json格式标签输出按照如下格式: \`\`\`json{'$defs': {'Person': {'description': '个人信息', 'properties': {'name': {'description': '人的姓名', 'title': 'Name', 'type': 'string'}, 'height_in_meters': {'description': '人的身高（米）', 'title': 'Height In Meters', 'type': 'number'}}, 'required': ['name', 'height_in_meters'], 'title': 'Person', 'type': 'object'}}, 'description': '文本中所有人的信息', 'properties': {'people': {'description': '文本中所有人的信息', 'items': {'$ref': '#/$defs/Person'}, 'title': 'People', 'type': 'array'}}, 'required': ['people'], 'title': 'People', 'type': 'object'}
\`\`\`.一定要确保你的回答在\`\`\`json 和 \`\`\`之间.
Human: 大傻春是个身高一米六六的18岁纯情男大.


[{'$defs': {'Person': {'description': '个人信息',
    'properties': {'name': {'description': '人的姓名',
      'title': 'Name',
      'type': 'string'},
     'height_in_meters': {'description': '人的身高（米）',
      'title': 'Height In Meters',
      'type': 'number'},
     'age': {'description': '年龄', 'title': 'Age', 'type': 'number'},
     'characteristics': {'description': '人物特征',
      'title': 'Characteristics',
      'type': 'string'}},
    'required': ['name', 'height_in_meters'],
    'title': 'Person',
    'type': 'object'}},
  'description': '文本中所有人的信息',
  'properties': {'people': {'description': '文本中所有人的信息',
    'items': {'$ref': '#/$defs/Person'},
    'title': 'People',
    'type': 'array'}},
  'required': ['people'],
  'title': 'People',
  'type': 'object'}]

In [33]:
chain.invoke({"query":query})

[{'people': [{'name': '大傻春', 'height_in_meters': 1.66}]}]